# Double Debiased Machine Learning for Continuous Treatments

Prof. Daniel de Abreu Pereira Uhr

## Conteúdo

* Double Debiased Machine Learning for Continuous Treatments - DDML for Continuous Treatments


## Referências

**Principais:**
* Colangelo and Lee (2023). Double Debiased Machine Learning Nonparametric Inference with Continuous Treatments. [arXiv:2004.03036v8 ](https://doi.org/10.48550/arXiv.2004.03036) 
* GitHub: https://github.com/KColangelo/Double-ML-Continuous-Treatment
* 
**Complementares:**
* Microsoft EconML: https://econml.azurewebsites.net/
* UBER CausalML: https://causalml.readthedocs.io/en/latest/
* https://docs.doubleml.org/stable/index.html
* https://github.com/MasaAsami/ReproducingDMLDiD/blob/main/notebook/Reproduction_of_DMLDiD_RO_for_NEW_SIMDATA.ipynb


# Double Debiased Machine Learning for Continuous Treatments

## Introdução

O **Double Debiased Machine Learning (DML)** é um método estatístico robusto e eficiente para estimar efeitos causais em contextos onde o tratamento é uma variável contínua. Este método combina:
- **Momentos Duplamente Robustos**, que garantem consistência mesmo quando uma das funções auxiliares é mal especificada.
- **Cross-fitting**, para evitar viés de overfitting causado pelo uso do mesmo conjunto de dados para treinamento e inferência.
- **Métodos de Machine Learning (ML)**, como LASSO, Redes Neurais ou Random Forests, para modelar funções de expectativa condicional e densidades condicionais.

O método é especialmente adequado para cenários onde o número de covariáveis é grande (alta dimensionalidade), sendo robusto a especificações incorretas de uma das funções auxiliares.

---

## Estrutura do Problema

### Dados e Objetivos

Considere uma amostra $\{Y_i, T_i, X_i\}_{i=1}^n$, onde:
- $Y_i$: Desfecho de interesse (variável dependente).
- $T_i$: Tratamento contínuo.
- $X_i$: Covariáveis observadas (potencialmente de alta dimensionalidade).

O objetivo principal é estimar:
1. **Função de Resposta Média à Dose (Average Dose-Response Function - ADRF)**:
   $$
   \beta_t = \mathbb{E}[Y(t)],
   $$
   onde $Y(t)$ representa o desfecho potencial para um valor específico $t$ do tratamento.

2. **Efeito Marginal (Partial Effect)**:
   $$
   \theta_t = \frac{\partial \beta_t}{\partial t}.
   $$

---

## Suposições para Identificação

Para identificar os efeitos causais, assumimos:
1. **Independência Condicional (Unconfoundedness)**:
   $$
   T \perp \varepsilon \mid X,
   $$
   onde $\varepsilon$ é o erro não observado. Esta suposição implica que, condicional nas covariáveis $X$, o tratamento é independentemente alocado.

2. **Suporte Comum (Common Support)**:
   $$
   f_{T|X}(t \mid X) > 0, \quad \forall t \in \mathcal{T}.
   $$
   Esta condição assegura que há sobreposição suficiente entre grupos de tratamento.

---

## Estimador DML

O estimador baseia-se em uma função momento duplamente robusta, definida como:
$$
\psi_t(Y_i, T_i, X_i) = \gamma(t, X_i) + \frac{K_h(T_i - t)}{f_{T|X}(t \mid X_i)} \left(Y_i - \gamma(t, X_i)\right),
$$
onde:
- $\gamma(t, X) = \mathbb{E}[Y \mid T = t, X]$: Expectativa condicional.
- $f_{T|X}(t \mid X)$: Densidade condicional do tratamento.
- $K_h(T_i - t)$: Função kernel para ponderar observações próximas do valor de tratamento $t$.

O estimador para $\beta_t$ é dado por:
$$
\hat{\beta}_t = \frac{1}{n} \sum_{i=1}^n \psi_t(Y_i, T_i, X_i).
$$

### Cross-Fitting
Para evitar viés de overfitting, os dados são particionados em $L$ subconjuntos (folds). Para cada fold $\ell$, as funções $\gamma$ e $f_{T|X}$ são estimadas usando apenas os dados fora do fold $\ell$. O estimador final é obtido pela média das estimativas em cada fold:
$$
\hat{\beta}_t = \frac{1}{n} \sum_{i=1}^n \left[ \gamma_{-\ell}(t, X_i) + \frac{K_h(T_i - t)}{f_{T|X,-\ell}(t \mid X_i)} \left(Y_i - \gamma_{-\ell}(t, X_i)\right) \right].
$$

---

## Propriedades Assintóticas

Sob condições regulares, o estimador $\hat{\beta}_t$ é:
1. **Consistente**: Converge para o valor verdadeiro $\beta_t$.
2. **Assintoticamente Normal**:
   $$
   \sqrt{n} (\hat{\beta}_t - \beta_t) \xrightarrow{d} N(0, V_t),
   $$
   onde $V_t$ é a variância assintótica, que pode ser estimada como:
   $$
   \hat{V}_t = \frac{1}{n^2} \sum_{i=1}^n \psi_t^2(Y_i, T_i, X_i).
   $$

---

## Estimação do Efeito Marginal

O efeito marginal $\theta_t$ é estimado numericamente:
$$
\hat{\theta}_t = \frac{\hat{\beta}_{t + \eta/2} - \hat{\beta}_{t - \eta/2}}{\eta},
$$
onde $\eta$ é uma sequência positiva que converge para zero à medida que $n \to \infty$.

Para garantir consistência e eficiência, $\eta$ deve ser escolhida adequadamente, levando em conta o tamanho amostral e a variabilidade nas estimativas de $\beta_t$.

---

## Benefícios e Limitações

### Benefícios
- **Flexibilidade**: Permite o uso de métodos de aprendizado de máquina para modelar $\gamma$ e $f_{T|X}$.
- **Eficiência**: Utiliza técnicas como cross-fitting para melhorar a precisão das estimativas.
- **Robustez**: Resistente a erros de especificação em uma das funções auxiliares.

### Limitações
- **Complexidade Computacional**: Requer a estimativa de funções auxiliares de alta dimensionalidade.
- **Sensibilidade ao Kernel**: A escolha do kernel e da largura de banda ($h$) pode impactar os resultados.

---

## Exemplo Intuitivo

Considere um estudo que analisa o impacto de **horas de treinamento em um programa de capacitação** ($T$) nos **salários futuros** ($Y$):
- Usamos $\gamma(t, X)$ para prever salários dados $T$ e as covariáveis $X$ (e.g., idade, escolaridade).
- Estimamos $f_{T|X}(t \mid X)$, que captura a distribuição das horas de treinamento, dado o perfil do indivíduo.
- O DML ajusta as estimativas para isolar o impacto causal de $T$ em $Y$, mesmo que algumas relações sejam complexas ou não lineares.

---

## Considerações Finais
O Double Debiased Machine Learning é um método avançado para estimar efeitos causais com tratamentos contínuos, oferecendo:

* Robustez a especificações incorretas de modelos auxiliares.
* Flexibilidade no uso de métodos de ML para modelagem de alta dimensionalidade.
* Garantias teóricas de consistência e normalidade assintótica.

Este método é ideal para aplicações empíricas que exigem alto rigor estatístico, como estudos em economia, saúde e políticas públicas.

---
